# Correlation matrix and visualization of variables and GDP

### This is the master notebook, please copy and create a new one for each group.

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro

In [10]:
df= pd.read_csv (os.getcwd()+'/Data/'+'Asia.csv')
df

,Unnamed: 0,Country,Year,Alcohol per capita,Education GExp,Employment-agriculture,Employment-industry,Employment-services,Exports-Commercial services,Exports-G&S,...,International taxes,Literacy rate,Mortality-infants,Net migration,Ninis,R&D GExp,Renewable electricity,Suicide,Workers high education,Continent
0,126,BGD,1990,1.000000,0.163503,1.000000,0.329289,0.000000,0.024443,0.000000,...,0.000000,0.000000,1.000000,0.962959,0.73922,NaN,NaN,1.000000,1.000000,Asia
1,127,BGD,1991,1.000000,0.152508,1.000000,0.329289,0.000000,0.040912,0.004397,...,0.000000,0.000000,0.949009,0.962959,0.73922,NaN,NaN,1.000000,1.000000,Asia
2,128,BGD,1992,1.000000,0.278258,0.980455,0.253900,0.063059,0.057294,0.012107,...,0.000000,0.030731,0.897379,0.962959,0.73922,NaN,NaN,1.000000,1.000000,Asia
3,129,BGD,1993,1.000000,0.242200,0.955463,0.181976,0.131657,0.071570,0.025255,...,0.000000,0.061462,0.846142,0.970367,0.73922,NaN,NaN,1.000000,1.000000,Asia
4,130,BGD,1994,1.000000,0.307929,0.926626,0.111785,0.204516,0.066132,0.026356,...,0.000000,0.092192,0.795503,0.977775,0.73922,NaN,NaN,1.000000,1.000000,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,782,KOR,2017,0.128788,NaN,0.000000,0.111111,0.977891,0.712108,0.915582,...,0.707104,NaN,0.023400,0.000000,NaN,0.906257,1.0,0.438710,1.000000,Asia
218,783,KOR,2018,0.000000,NaN,0.008256,0.101010,0.977891,0.841297,1.000000,...,0.674260,NaN,0.015052,0.000000,NaN,1.000000,1.0,0.619355,0.986142,Asia
219,784,KOR,2019,0.000000,NaN,0.022704,0.011544,1.000000,0.841531,0.890730,...,0.594235,NaN,0.007210,0.000000,NaN,1.000000,1.0,0.619355,1.000000,Asia
220,785,KOR,2020,0.000000,NaN,0.022704,0.011544,1.000000,0.713233,0.811081,...,0.594235,NaN,0.000000,0.000000,NaN,1.000000,1.0,0.619355,0.782909,Asia


## Visualization

In [11]:
columns=['Country','Year','Exports-Commercial services','Renewable electricity','Employment-agriculture','Employment-industry','Employment-services','Exports-G&S','Fertility rate','Foreign investment','GDP','Education GExp','Workers high education','Literacy rate','Net migration','Mortality-infants','Health services use','R&D GExp','Ninis','Suicide','International taxes','Alcohol per capita']
clist=df['Country'].unique()

The following loop has been created to test if each variable for each country follows a normal distribution and also to present in an orderly manner all the correlations between the GDP and the variables so they can be understood on a lower scale.

In [12]:
def multcolumn(frame):
    for u in range(3, len(columns)):
        name2=columns[u]+'.^2'
        name3=columns[u]+'.^3'
        namelog=columns[u]+'.log'
        frame.loc[:,name2] = frame[columns[u]]**2
        frame.loc[:,name3] = frame[columns[u]]**3
        frame.loc[:,namelog] = np.log(frame[columns[u]])

In [13]:
multcolumn(df)
for i in range(0,len(clist)):
    dat=df.loc[df.loc[:, 'Country'] == clist[i]]
    for e in range(3,len(columns)):
        data=dat.iloc[:, e]
        stat, p = shapiro(data)
        print(clist[i] +"-"+ columns[e])
        print('Statistical=%.3f, p=%.3f' % (stat, p))
        alpha = 0.05
        if p > alpha:
            print('Data is NORMAL ( H0 not denied )')
        else:
            print('It does NOT seem normal( Denies H0 )')
    print(clist[i])
    cor=dat.corr()    
    cor.loc[:,'GDP-R^2'] = cor['GDP']**2  
    cor.loc[:,'Indicator']=cor.index
    cor[['Indicator','Type']]=cor.Indicator.str.split('.',expand=True)  
    corcolumn=cor[['Indicator','Type','GDP-R^2','GDP']]
    corcolumn=corcolumn.loc[corcolumn.loc[:, 'GDP-R^2'] >= 0.75]
    id=corcolumn.groupby('Indicator')['GDP-R^2'].transform(max)==corcolumn['GDP-R^2']
    corcolumn[id]
    max_df=pd.DataFrame(corcolumn[id])
    max_df['Behaviour']=np.where(max_df['GDP']>0, 'Positive', 'Negative')
    max_df['Type']=max_df['Type'].replace(['^2','^3','log'],['Cuadratic','Cubic','Logarithmic'])
    max_df.drop("GDP",axis=1,inplace=True)
    max_df=max_df.reset_index(drop=True)
    max_df = max_df.drop(max_df[max_df['Indicator']=='Year'].index)
    max_df = max_df.drop(max_df[max_df['Indicator']=='GDP'].index)
    max_df = max_df.drop(max_df[max_df['Indicator']=='Unnamed: 0'].index)
    max_df=max_df.sort_values(by = 'GDP-R^2',ascending = False).style.background_gradient()
    display(max_df)

BGD-Renewable electricity
Statistical=0.808, p=0.000
It does NOT seem normal( Denies H0 )
BGD-Employment-agriculture
Statistical=0.978, p=0.734
Data is NORMAL ( H0 not denied )
BGD-Employment-industry
Statistical=0.885, p=0.003
It does NOT seem normal( Denies H0 )
BGD-Employment-services
Statistical=0.899, p=0.006
It does NOT seem normal( Denies H0 )
BGD-Exports-G&S
Statistical=0.895, p=0.005
It does NOT seem normal( Denies H0 )
BGD-Fertility rate
Statistical=0.827, p=0.000
It does NOT seem normal( Denies H0 )
BGD-Foreign investment
Statistical=0.855, p=0.001
It does NOT seem normal( Denies H0 )
BGD-GDP
Statistical=0.904, p=0.008
It does NOT seem normal( Denies H0 )
BGD-Education GExp
Statistical=0.862, p=0.001
It does NOT seem normal( Denies H0 )
BGD-Workers high education
Statistical=0.814, p=0.000
It does NOT seem normal( Denies H0 )
BGD-Literacy rate
Statistical=0.853, p=0.000
It does NOT seem normal( Denies H0 )
BGD-Net migration
Statistical=0.761, p=0.000
It does NOT seem normal(

c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\amarchve\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zer

,Indicator,Type,GDP-R^2,Behaviour
16,Health services use,Cuadratic,0.982772,Positive
10,Employment-industry,Cubic,0.979148,Positive
14,Literacy rate,Cuadratic,0.978476,Positive
6,International taxes,None,0.975595,Positive
3,Exports-Commercial services,None,0.960355,Positive
12,Fertility rate,Logarithmic,0.958683,Negative
4,Exports-G&S,None,0.958007,Positive
15,Mortality-infants,Logarithmic,0.957629,Negative
2,Alcohol per capita,None,0.932819,Negative
9,Employment-agriculture,Logarithmic,0.926290,Negative


IND-Renewable electricity
Statistical=0.743, p=0.000
It does NOT seem normal( Denies H0 )
IND-Employment-agriculture
Statistical=0.892, p=0.005
It does NOT seem normal( Denies H0 )
IND-Employment-industry
Statistical=0.892, p=0.005
It does NOT seem normal( Denies H0 )
IND-Employment-services
Statistical=0.843, p=0.000
It does NOT seem normal( Denies H0 )
IND-Exports-G&S
Statistical=0.922, p=0.027
It does NOT seem normal( Denies H0 )
IND-Fertility rate
Statistical=0.848, p=0.000
It does NOT seem normal( Denies H0 )
IND-Foreign investment
Statistical=0.832, p=0.000
It does NOT seem normal( Denies H0 )
IND-GDP
Statistical=0.930, p=0.044
It does NOT seem normal( Denies H0 )
IND-Education GExp
Statistical=0.829, p=0.000
It does NOT seem normal( Denies H0 )
IND-Workers high education
Statistical=0.858, p=0.001
It does NOT seem normal( Denies H0 )
IND-Literacy rate
Statistical=0.854, p=0.001
It does NOT seem normal( Denies H0 )
IND-Net migration
Statistical=0.884, p=0.003
It does NOT seem nor

,Indicator,Type,GDP-R^2,Behaviour
5,Exports-Commercial services,None,0.988543,Positive
8,Health services use,None,0.987243,Positive
9,Literacy rate,None,0.980034,Positive
3,Employment-agriculture,None,0.971213,Negative
4,Employment-services,None,0.965172,Positive
6,Exports-G&S,None,0.960976,Positive
14,Fertility rate,Logarithmic,0.957240,Negative
10,Renewable electricity,None,0.956726,Positive
13,Employment-industry,Cuadratic,0.956689,Positive
2,Alcohol per capita,None,0.951519,Positive


...

,Indicator,Type,GDP-R^2,Behaviour
18,Suicide,Logarithmic,0.961053,Negative
9,Health services use,None,0.957287,Positive
11,Literacy rate,None,0.940276,Positive
13,Renewable electricity,Cubic,0.935800,Positive
5,Exports-G&S,None,0.922161,Positive
15,Employment-services,Cuadratic,0.895565,Positive
12,Ninis,None,0.873700,Negative
7,Foreign investment,None,0.866991,Negative
10,International taxes,None,0.860822,Positive
2,Alcohol per capita,None,0.844143,Positive


PHL-Renewable electricity
Statistical=0.845, p=0.000
It does NOT seem normal( Denies H0 )
PHL-Employment-agriculture
Statistical=0.960, p=0.271
Data is NORMAL ( H0 not denied )
PHL-Employment-industry
Statistical=0.946, p=0.112
Data is NORMAL ( H0 not denied )
PHL-Employment-services
Statistical=0.900, p=0.006
It does NOT seem normal( Denies H0 )
PHL-Exports-G&S
Statistical=0.945, p=0.106
Data is NORMAL ( H0 not denied )
PHL-Fertility rate
Statistical=0.880, p=0.002
It does NOT seem normal( Denies H0 )
PHL-Foreign investment
Statistical=0.943, p=0.089
Data is NORMAL ( H0 not denied )
PHL-GDP
Statistical=0.943, p=0.089
Data is NORMAL ( H0 not denied )
PHL-Education GExp
Statistical=0.981, p=0.832
Data is NORMAL ( H0 not denied )
PHL-Workers high education
Statistical=0.849, p=0.000
It does NOT seem normal( Denies H0 )
PHL-Literacy rate
Statistical=0.805, p=0.000
It does NOT seem normal( Denies H0 )
PHL-Net migration
Statistical=0.856, p=0.001
It does NOT seem normal( Denies H0 )
PHL-Mor

,Indicator,Type,GDP-R^2,Behaviour
8,International taxes,None,0.980667,Positive
7,Health services use,None,0.972626,Positive
11,Employment-services,Cuadratic,0.949321,Positive
3,Exports-Commercial services,None,0.942273,Positive
4,Exports-G&S,None,0.940955,Positive
5,Fertility rate,None,0.940254,Negative
10,Mortality-infants,None,0.869246,Negative
2,Employment-agriculture,None,0.864061,Negative
9,Literacy rate,None,0.824624,Positive
13,Alcohol per capita,Cuadratic,0.809338,Positive


KOR-Renewable electricity
Statistical=0.778, p=0.000
It does NOT seem normal( Denies H0 )
KOR-Employment-agriculture
Statistical=nan, p=1.000
Data is NORMAL ( H0 not denied )
KOR-Employment-industry
Statistical=0.910, p=0.011
It does NOT seem normal( Denies H0 )
KOR-Employment-services
Statistical=0.819, p=0.000
It does NOT seem normal( Denies H0 )
KOR-Exports-G&S
Statistical=0.864, p=0.001
It does NOT seem normal( Denies H0 )
KOR-Fertility rate
Statistical=0.891, p=0.004
It does NOT seem normal( Denies H0 )
KOR-Foreign investment
Statistical=0.870, p=0.001
It does NOT seem normal( Denies H0 )
KOR-GDP
Statistical=0.953, p=0.176
Data is NORMAL ( H0 not denied )
KOR-Education GExp
Statistical=0.904, p=0.008
It does NOT seem normal( Denies H0 )
KOR-Workers high education
Statistical=0.914, p=0.014
It does NOT seem normal( Denies H0 )
KOR-Literacy rate
Statistical=0.731, p=0.000
It does NOT seem normal( Denies H0 )
KOR-Net migration
Statistical=0.963, p=0.335
Data is NORMAL ( H0 not denied

,Indicator,Type,GDP-R^2,Behaviour
12,Mortality-infants,Logarithmic,0.939542,Negative
10,Workers high education,None,0.938016,Positive
8,R&D GExp,None,0.927730,Positive
5,Exports-G&S,None,0.924029,Positive
4,Exports-Commercial services,None,0.917982,Positive
11,Employment-services,Cubic,0.910248,Positive
3,Employment-agriculture,None,0.902786,Negative
9,Renewable electricity,None,0.878281,Positive
2,Alcohol per capita,None,0.872339,Negative
13,Health services use,Cubic,0.839002,Negative
